### Exercise 1.1: A baseline MLP

#### Imports and Dataset

In [14]:
import numpy as np
import torch
from torchvision.datasets import MNIST
from torch.utils.data import Subset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from dataclasses import dataclass
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from tqdm import tqdm

In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

ds_train = MNIST(root='./data', train=True, download=True, transform=transform)
ds_test = MNIST(root='./data', train=False, download=True, transform=transform)

val_size = 5000
I = np.random.permutation(len(ds_train))
ds_val = Subset(ds_train, I[:val_size])
ds_train = Subset(ds_train, I[val_size:])

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


#### MLP e Configuration

In [17]:
@dataclass
class Config:
    batch_size: int = 128
    learning_rate: float = 1e-3
    epochs: int = 10
    hidden_layer_sizes: list = None
    log_dir: str = "runs/mnist_mlp_experiment"
    device: torch.device = device
    input_size: int = 28*28
    num_classes: int = 10
    activation_class: type = nn.ReLU   
    name: str = None                   

class MLP(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes, num_classes, activation_class=nn.ReLU):
        super().__init__()
        layers = [nn.Flatten()]

        for hidden_layer_size in hidden_layer_sizes:
            layers.append(nn.Linear(input_size, hidden_layer_size))
            layers.append(activation_class())
            input_size = hidden_layer_size

        layers.append(nn.Linear(input_size, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

#### Training utilities

In [19]:
def train_one_epoch(model, dataloader, optimizer, criterion, device, epoch=None):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    loop = tqdm(dataloader, desc=f"Epoch {epoch+1}" if epoch is not None else "Train", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        running_corrects += (preds == labels).sum().item()
        total_samples += labels.size(0)

        loop.set_postfix(loss=running_loss/total_samples, acc=running_corrects/total_samples)

    average_loss = running_loss / total_samples
    accuracy = running_corrects / total_samples
    return average_loss, accuracy


def evaluate(model, dataloader, criterion, device, split_name="Test"):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    loop = tqdm(dataloader, desc=split_name, leave=False)
    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            running_corrects += (preds == labels).sum().item()
            total_samples += labels.size(0)

            loop.set_postfix(loss=running_loss/total_samples, acc=running_corrects/total_samples)

    average_loss = running_loss / total_samples
    accuracy = running_corrects / total_samples
    return average_loss, accuracy


def assign_name(config):
    config.name = f"MLP_{'-'.join(map(str, config.hidden_layer_sizes))}"
    return config


In [20]:
def run_experiments(configs):
    results = []

    for config in configs:
        config = assign_name(config)
        print(f"\nRunning experiment: {config.name}")

        train_loader = torch.utils.data.DataLoader(ds_train, batch_size=config.batch_size, shuffle=True)
        test_loader  = torch.utils.data.DataLoader(ds_test, batch_size=config.batch_size, shuffle=False)

        model = MLP(input_size=config.input_size,
                    hidden_layer_sizes=config.hidden_layer_sizes,
                    num_classes=config.num_classes,
                    activation_class=config.activation_class).to(config.device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
        writer = SummaryWriter(log_dir=f"{config.log_dir}/{config.name}")

        best_test_acc = 0.0
        best_epoch = 0

        for epoch in range(config.epochs):
            train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, config.device, epoch)
            test_loss, test_acc = evaluate(model, test_loader, criterion, config.device, split_name="Test")

            writer.add_scalar("Loss/train", train_loss, epoch)
            writer.add_scalar("Accuracy/train", train_acc, epoch)
            writer.add_scalar("Loss/test", test_loss, epoch)
            writer.add_scalar("Accuracy/test", test_acc, epoch)

            if test_acc > best_test_acc:
                best_test_acc = test_acc
                best_epoch = epoch + 1  

        writer.close()

        print(f"Model {config.name} | Best Test Acc: {best_test_acc:.4f} reached at epoch {best_epoch}")

        results.append({
            "model": config.name,
            "best_test_acc": best_test_acc,
            "best_epoch": best_epoch
        })

    df = pd.DataFrame(results)
    print("\nSummary of best test accuracies per model")
    print(df)
    return df

In [21]:
configs = [
    Config(hidden_layer_sizes=[16,16], epochs=11),
    Config(hidden_layer_sizes=[64,32,16], epochs=11),
    Config(hidden_layer_sizes=[256,128], epochs=11),
]

summary_df = run_experiments(configs)


Running experiment: MLP_16-16


Model MLP_16-16 | Best Test Acc: 0.9541 reached at epoch 11

Running experiment: MLP_64-32-16


Model MLP_64-32-16 | Best Test Acc: 0.9712 reached at epoch 11

Running experiment: MLP_256-128


Model MLP_256-128 | Best Test Acc: 0.9816 reached at epoch 10

Summary of best test accuracies per model
          model  best_test_acc  best_epoch
0     MLP_16-16         0.9541          11
1  MLP_64-32-16         0.9712          11
2   MLP_256-128         0.9816          10


### Exercise 1.2: Adding Residual Connections

Implement a variant of your parameterized MLP network to support **residual** connections. Your network should be defined as a composition of **residual MLP** blocks that have one or more linear layers and add a skip connection from the block input to the output of the final linear layer.

**Compare** the performance (in training/validation loss and test accuracy) of your MLP and ResidualMLP for a range of depths. Verify that deeper networks **with** residual connections are easier to train than a network of the same depth **without** residual connections.

**For extra style points**: See if you can explain by analyzing the gradient magnitudes on a single training batch *why* this is the case. 

In [8]:
# Your code here.

### Exercise 1.3: Rinse and Repeat (but with a CNN)

Repeat the verification you did above, but with **Convolutional** Neural Networks. If you were careful about abstracting your model and training code, this should be a simple exercise. Show that **deeper** CNNs *without* residual connections do not always work better and **even deeper** ones *with* residual connections.

**Hint**: You probably should do this exercise using CIFAR-10, since MNIST is *very* easy (at least up to about 99% accuracy).

**Tip**: Feel free to reuse the ResNet building blocks defined in `torchvision.models.resnet` (e.g. [BasicBlock](https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py#L59) which handles the cascade of 3x3 convolutions, skip connections, and optional downsampling). This is an excellent exercise in code diving. 

**Spoiler**: Depending on the optional exercises you plan to do below, you should think *very* carefully about the architectures of your CNNs here (so you can reuse them!).

In [9]:
# Your code here.

-----
## Exercise 2: Choose at Least One

Below are **three** exercises that ask you to deepen your understanding of Deep Networks for visual recognition. You must choose **at least one** of the below for your final submission -- feel free to do **more**, but at least **ONE** you must submit. Each exercise is designed to require you to dig your hands **deep** into the guts of your models in order to do new and interesting things.

**Note**: These exercises are designed to use your small, custom CNNs and small datasets. This is to keep training times reasonable. If you have a decent GPU, feel free to use pretrained ResNets and larger datasets (e.g. the [Imagenette](https://pytorch.org/vision/0.20/generated/torchvision.datasets.Imagenette.html#torchvision.datasets.Imagenette) dataset at 160px).

### Exercise 2.1: *Fine-tune* a pre-trained model
Train one of your residual CNN models from Exercise 1.3 on CIFAR-10. Then:
1. Use the pre-trained model as a **feature extractor** (i.e. to extract the feature activations of the layer input into the classifier) on CIFAR-100. Use a **classical** approach (e.g. Linear SVM, K-Nearest Neighbor, or Bayesian Generative Classifier) from scikit-learn to establish a **stable baseline** performance on CIFAR-100 using the features extracted using your CNN.
2. Fine-tune your CNN on the CIFAR-100 training set and compare with your stable baseline. Experiment with different strategies:
    - Unfreeze some of the earlier layers for fine-tuning.
    - Test different optimizers (Adam, SGD, etc.).

Each of these steps will require you to modify your model definition in some way. For 1, you will need to return the activations of the last fully-connected layer (or the global average pooling layer). For 2, you will need to replace the original, 10-class classifier with a new, randomly-initialized 100-class classifier.

In [10]:
# Your code here.

### Exercise 2.2: *Distill* the knowledge from a large model into a smaller one
In this exercise you will see if you can derive a *small* model that performs comparably to a larger one on CIFAR-10. To do this, you will use [Knowledge Distillation](https://arxiv.org/abs/1503.02531):

> Geoffrey Hinton, Oriol Vinyals, and Jeff Dean. Distilling the Knowledge in a Neural Network, NeurIPS 2015.

To do this:
1. Train one of your best-performing CNNs on CIFAR-10 from Exercise 1.3 above. This will be your **teacher** model.
2. Define a *smaller* variant with about half the number of parameters (change the width and/or depth of the network). Train it on CIFAR-10 and verify that it performs *worse* than your **teacher**. This small network will be your **student** model.
3. Train the **student** using a combination of **hard labels** from the CIFAR-10 training set (cross entropy loss) and **soft labels** from predictions of the **teacher** (Kulback-Leibler loss between teacher and student).

Try to optimize training parameters in order to maximize the performance of the student. It should at least outperform the student trained only on hard labels in Setp 2.

**Tip**: You can save the predictions of the trained teacher network on the training set and adapt your dataloader to provide them together with hard labels. This will **greatly** speed up training compared to performing a forward pass through the teacher for each batch of training.

In [11]:
# Your code here.

### Exercise 2.3: *Explain* the predictions of a CNN

Use the CNN model you trained in Exercise 1.3 and implement [*Class Activation Maps*](http://cnnlocalization.csail.mit.edu/#:~:text=A%20class%20activation%20map%20for,decision%20made%20by%20the%20CNN.):

> B. Zhou, A. Khosla, A. Lapedriza, A. Oliva, and A. Torralba. Learning Deep Features for Discriminative Localization. CVPR'16 (arXiv:1512.04150, 2015).

Use your CNN implementation to demonstrate how your trained CNN *attends* to specific image features to recognize *specific* classes. Try your implementation out using a pre-trained ResNet-18 model and some images from the [Imagenette](https://pytorch.org/vision/0.20/generated/torchvision.datasets.Imagenette.html#torchvision.datasets.Imagenette) dataset -- I suggest you start with the low resolution version of images at 160px.

In [12]:
# Your code here.